In [1]:
import pandas as pd

In [2]:
# assuming ncaa2019 package is installed 
from ncaa2019 import DataSet

In [3]:
ds = DataSet()

default path used C:\Users\Huang\kaggle-ncaa-2019-data


In [4]:
ds.list_raw_keys()

['city',
 'conf',
 'conftour',
 'gamecity',
 'massey',
 't_compact_result',
 't_detail_result',
 't_seed_slot',
 't_seed',
 't_slot',
 'r_compact_result',
 'r_detail_result',
 'season',
 'st_compact_result',
 'st_team',
 'team',
 'coache',
 'team_conf',
 'team_spelling']

# Team Score Difference

In [5]:
regular_bs = ds.get_raw_data('r_compact_result')

In [6]:
regular_bs['DayNum'].max()

132

In [7]:
regular_bs.head().style

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [8]:
team_score_season_hist = pd.concat([
    (regular_bs
     .assign(relative_score = lambda x: x['WScore'] - x['LScore'],
             TeamID = lambda x: x['WTeamID'])
     .loc[:,['Season', 'TeamID', 'relative_score']]
    ),
    (regular_bs
     .assign(relative_score = lambda x: x['LScore'] - x['WScore'],
             TeamID = lambda x: x['LTeamID'])
     .loc[:,['Season', 'TeamID', 'relative_score']]
    )],
    ignore_index=True,
    axis=0)
    

In [9]:
team_score_season_avg = team_score_season_hist.groupby(['Season', 'TeamID'])[['relative_score']].mean()
team_score_season_avg.head()

relative_score
Season TeamID                
1985   1102         -5.791667
       1103         -3.043478
       1104          7.800000
       1106         -3.791667
       1108          7.960000

In [10]:
team_score_season_avg.columns

Index(['relative_score'], dtype='object')

# Rank Feature

In [11]:
rank_df = ds.get_raw_data('massey')

In [12]:
from scipy import stats

In [79]:
rank_df['RankingDayNum'].max()

133

In [82]:
# check if rank only includes regular season
assert(rank_df['RankingDayNum'].max()<134)

In [13]:
rank_df.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [89]:
rank_cor_df = rank_df.set_index(['Season', 'TeamID', 'RankingDayNum', 'SystemName'])['OrdinalRank'].unstack(-1).corr()

In [93]:
rank_cor_df.fillna(0).style

SystemName,7OT,ACU,ADE,AP,ARG,AUS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BNT,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,COX,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DC2,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INP,ISR,JCI,JEN,JJK,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LAB,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGY,MIC,MKV,MMG,MOR,MPI,MSX,MUZ,MvG,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STM,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WMR,WMV,WOB,WOL,WTE,YAG,ZAM
SystemName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7OT,1,0.949793,0.912851,0.477064,0.951922,0,0.955928,0,0,0.941982,0,0.948427,0.912703,0.770847,0.950616,0.931731,0,0.964691,0,0.948861,0.953266,0.947208,0,0.951335,0.947907,0.945276,0.955382,0.919055,0.952447,0.971219,0.955536,0.591946,0.951139,0.953797,0.954985,0.938951,0.907675,0.451211,0.946932,0.938861,0.936863,0.936016,0,0,0.95675,0,0,0,0.938112,0.957833,0.954597,0.924367,0,0,0.957645,0.933696,0.953364,0,0.951824,0.951368,0,0.864505,0,0.934944,0,0,0.936302,0.951324,0.925576,0,0.945529,0.909758,0,0,0,0.947676,0.940699,0.942812,0.948243,0.955246,0.942649,0.926718,0,0.924274,0.944685,0.944576,0.921331,0,0,0,0.944009,0.920829,0.930207,0.947977,0.947209,0.958016,0.931992,0,0.950844,0.960661,0,0.939864,0,0.919975,0,0.956794,0.944288,0.955021,0.952163,0.914941,0,0.940719,0,0.943532,0,0,0.935874,0,0.940471,0,0.933191,0.947895,0.932804,0.949924,0.939116,0.91337,0.9475,0,0,0,0.930315,0.950448,0.930842,0.961273,0,0.929256,0.95756,0.945723,0.952945,0.944169,0.949056,0.961386,0.954936,0,0.941142,0.95247,0.934963,0,0.942667,0.943854,0.940573,0,0,0.820318,0,0.951367,0.478563,0.91433,0.954227,0.921032,0.95674,0.948607,0.936713,0,0.945791,0.949802
ACU,0.949793,1,0.9743,0.713269,0.971185,0,0.971765,0.854766,0.838991,0.959411,0,0.983591,0,0,0.973806,0.975195,0,0,0,0.98269,0.988646,0,0.693711,0.964937,0.959124,0,0.972825,0.952195,0.981239,0,0.982965,0.69882,0.973545,0.953171,0,0.973391,0,0.634234,0.981996,0.964069,0.958286,0.964436,0.876464,0,0.985056,0.856677,0.851839,0,0,0.995626,0,0.928336,0,0.869849,0,0.963603,0,0.849777,0.991607,0,0,0.919551,0,0,0.929664,0.891787,0,0,0,0,0,0.885799,0,0.839947,0,0,0.980732,0.97093,0.980172,0,0.971999,0.938034,0.868663,0.904453,0.979412,0.890745,0.968918,0,0,0,0,0.957885,0.932618,0.957894,0,0.994783,0.93494,0,0,0,0,0.974441,0.901984,0.956271,0.859145,0,0.965075,0,0,0,0.755725,0,0.902919,0.975943,0.85217,0,0,0.901089,0.936765,0,0,0.956115,0,0.978946,0.957267,0.948013,0.966683,0.892645,0,0.866322,0.936888,0.978231,0.965289,0,0.830412,0,0,0.980425,0.985634,0.956653,0.978301,0.989722,0,0,0.963636,0,0.967221,0,0.975576,0.982064,0.980414,0.799132,0.870516,0.841883,0.909804,0,0.721754,0.955006,0.958121,0.969786,0,0.963965,0.94795,0,0,0
ADE,0.912851,0.9743,1,0.502636,0.962958,0,0.966516,0,0,0.968334,0,0.938947,0.928631,0,0.96219,0,0,0.987813,0,0.950487,0.949744,0.961266,0,0.942463,0.967421,0,0.957701,0.937009,0.954154,0.956537,0,0.697423,0.93306,0.963831,0.958284,0.945929,0.934663,0.584698,0.962349,0.933363,0.970238,0.922284,0,0.890519,0.959427,0,0,0,0.957724,0.951796,0.941408,0.910942,0,0,0.964455,0.913708,0.980005,0,0.959055,0.931261,0,0,0,0.925939,0,0,0.923793,0,0.831566,0,0.945318,0.905699,0,0,0,0.960158,0.961207,0.953992,0.969175,0,0.921629,0.938841,0,0.905822,0.960033,0.963554,0.931888,0,0,0,0,0.892784,0.942023,0.932225,0.925719,0.984067,0.946584,0,0.93638,0.943375,0,0.963172,0,0.921265,0,0.943018,0.929263,0.941075,0.940676,0.909651,0,0.920769,0,0.975741,0,0,0.961824,0,0.971586,0,0.891451,0.927286,0.970659,0.955608,0.95255,0.953939,0.947919,0,0,0,0.960269,0.965057,0.937325,0.947821,0,0.942548,0.956755,0.939506,0.948576,0.956101,0.963654,0.970612,0,0,0.970916,0.943291,0.965047,0,0.93

In [14]:
team_avg_rank = (rank_df
                 .groupby(['Season','TeamID'])[['OrdinalRank']]
                 .mean()
                )

In [15]:
team_avg_rank.head()

OrdinalRank
Season TeamID             
2003   1102     144.287500
       1103     182.205000
       1104      27.655502
       1105     305.377500
       1106     243.265000

# Labels

In [16]:
wl_records = pd.concat([
    (ds
     .get_raw_data('t_compact_result')
     .assign(GameType='T')
    ),
    (ds
    .get_raw_data('r_compact_result')
     .assign(GameType='R')
    )],
    ignore_index=True,
    axis=0)


In [17]:
wl_records_symmetric = pd.concat([
    (wl_records
     .rename(columns={'WTeamID':'TeamID1',
                      'LTeamID':'TeamID2'})
     .assign(Winner='Team1')
     .loc[:,['Season', 'GameType', 'TeamID1', 'TeamID2', 'Winner']]                     
    ),
    (wl_records
     .rename(columns={'LTeamID':'TeamID1',
                      'WTeamID':'TeamID2'})
     .assign(Winner='Team2')
     .loc[:,['Season', 'GameType', 'TeamID1', 'TeamID2', 'Winner']]                     
    )],
    ignore_index=True,
    axis=0
)

# Feature and Label Data Frame


In [18]:
feature_label = (
    pd.merge(
        pd.merge(
            wl_records_symmetric,
            (team_score_season_avg
             .reset_index()
             .rename(columns={'TeamID': 'TeamID1', 'relative_score':'Team1_season_score'})
            ),
            on=['TeamID1', 'Season']
        ),       
        (team_score_season_avg
         .reset_index()
         .rename(columns={'TeamID': 'TeamID2', 'relative_score':'Team2_season_score'})
        ),
        on=['TeamID2', 'Season']
    ))

Top 5 row of feature and label data frame

In [19]:
feature_label.head().style

,Season,GameType,TeamID1,TeamID2,Winner,Team1_season_score,Team2_season_score
0,1985,T,1116,1234,Team1,3.63636,10.4667
1,1985,R,1116,1234,Team2,3.63636,10.4667
2,1985,R,1326,1234,Team2,4.67857,10.4667
3,1985,R,1326,1234,Team2,4.67857,10.4667
4,1985,R,1228,1234,Team1,10.871,10.4667


# Training and Test Data
We only use season average WL difference as the feature.
All regular season result + Playoff before 2011 are available for training
Playoff after 2011 including 2011 are set aside as test data

In [20]:
from sklearn.utils import shuffle

In [21]:
def train_test_df_split(feature_label):
    train_data = feature_label.loc[(feature_label['Season']<2011) |
                                   (feature_label['GameType']=='R'), :]
    test_data = feature_label.loc[(feature_label['Season']>=2011) &
                                  (feature_label['GameType']=='T'), :]
    return train_data, test_data


def train_test_split(feature_label):
    train_df, test_df = train_test_df_split(feature_label)
    X_train = train_df[['Team1_season_score','Team2_season_score']].values
    y_train = train_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values
    X_test = test_df[['Team1_season_score','Team2_season_score']].values
    y_test = test_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values    
    return X_train, X_test, y_train, y_test

In [22]:
X_train, X_test, y_train, y_test = train_test_split(feature_label)
print(X_train.shape)

(315474, 2)


Top of train and test data

In [23]:
print(f"""
X_train:
{X_train}
y_train
{y_train}
""")



X_train:
[[  3.63636364  10.46666667]
 [  3.63636364  10.46666667]
 [  4.67857143  10.46666667]
 ...
 [-13.71875      0.26666667]
 [-13.71875      0.26666667]
 [-15.26666667   0.26666667]]
y_train
[1 0 0 ... 0 0 0]



# Logistic Regression
We apply the logistc regression to test the result

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
clf = LogisticRegression(fit_intercept=False)
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
import numpy as np

In [27]:
def KL_score(y_value, p_predict):
    score = -(np.sum(np.log(p_predict[y_value==1])) + np.sum(np.log(1-p_predict[y_value==0])))/len(y_value)
    return score

In [28]:
def predict_prob(clf, X):
    prob_class = clf.predict_proba(X)
    prob = prob_class[:,1]
    return prob

In [29]:
print(
f"""
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)


KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.5291835615488899
KL Score in testing data: 0.6194114258739689



# only use post season scores

In [30]:
feature_label.groupby('GameType').count()

,Season,TeamID1,TeamID2,Winner,Team1_season_score,Team2_season_score
GameType,,,,,,
R,312178,312178,312178,312178,312178,312178
T,4368,4368,4368,4368,4368,4368


In [31]:
def train_test_df_split(feature_label):
    train_data = feature_label.loc[(feature_label['Season']<2011) &
                                   (feature_label['GameType']=='T'), :]
    test_data = feature_label.loc[(feature_label['Season']>=2011) &
                                  (feature_label['GameType']=='T'), :]
    return train_data, test_data


def train_test_split(feature_label):
    train_df, test_df = train_test_df_split(feature_label)
    X_train = train_df[['Team1_season_score','Team2_season_score']].values
    y_train = train_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values
    X_test = test_df[['Team1_season_score','Team2_season_score']].values
    y_test = test_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values    
    return X_train, X_test, y_train, y_test

In [32]:
X_train, X_test, y_train, y_test = train_test_split(feature_label)

In [33]:
X_train.shape

(3296, 2)

In [34]:
clf = LogisticRegression(fit_intercept=False)
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [35]:
print(
f"""
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)


KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.5973824388937066
KL Score in testing data: 0.6180925730126637



In [36]:
from sklearn.linear_model import LogisticRegressionCV

In [37]:
from sklearn import metrics

In [38]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted'])

In [39]:
clf = LogisticRegressionCV(fit_intercept=False,scoring =  'neg_log_loss')
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='neg_log_loss',
           solver='lbfgs', tol=0.0001, verbose=0)

In [40]:
print(
f"""
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)


KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.5973984453130364
KL Score in testing data: 0.6179979325204425



# Rank + Score

In [41]:
feature_label = (
    pd.merge(
        pd.merge(
            wl_records_symmetric,
            (team_score_season_avg
             .reset_index()
             .rename(columns={'TeamID': 'TeamID1', 'relative_score':'Team1_season_score'})
            ),
            on=['TeamID1', 'Season']
        ),       
        (team_score_season_avg
         .reset_index()
         .rename(columns={'TeamID': 'TeamID2', 'relative_score':'Team2_season_score'})
        ),
        on=['TeamID2', 'Season']
    ))

In [42]:
score_rank = pd.merge(team_score_season_avg.reset_index(), team_avg_rank.reset_index(), on=['Season', 'TeamID'])

In [43]:
feature_label = (
    pd.merge(
        pd.merge(
            wl_records_symmetric,
            (score_rank             
             .rename(columns={'TeamID': 'TeamID1', 'relative_score':'Team1_season_score', 'OrdinalRank':'Team1_OrdinalRank'
                             })
            ),
            on=['TeamID1', 'Season']
        ),       
        (score_rank         
         .rename(columns={'TeamID': 'TeamID2', 'relative_score':'Team2_season_score', 'OrdinalRank':'Team2_OrdinalRank'})
        ),
        on=['TeamID2', 'Season']
    ))

In [64]:
def train_test_df_split(feature_label):
    train_data = feature_label.loc[(feature_label['Season']<2011) |
                                   (feature_label['GameType']=='R'), :]
    test_data = feature_label.loc[(feature_label['Season']>=2011) &
                                  (feature_label['GameType']=='T'), :]
    return train_data, test_data


def train_test_split(feature_label):
    train_df, test_df = train_test_df_split(feature_label)
    X_train = train_df[['Team1_season_score','Team2_season_score','Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_train = train_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values
    X_test = test_df[['Team1_season_score','Team2_season_score','Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_test = test_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values    
    return X_train, X_test, y_train, y_test

In [65]:
X_train, X_test, y_train, y_test = train_test_split(feature_label)

In [66]:
from sklearn import preprocessing
from sklearn import pipeline

In [67]:
clf = pipeline.Pipeline([('SS', preprocessing.StandardScaler()),
                ('LR', LogisticRegressionCV(fit_intercept=False,scoring =  'neg_log_loss'))])
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Pipeline(memory=None,
     steps=[('SS', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='neg_log_loss',
           solver='lbfgs', tol=0.0001, verbose=0))])

In [68]:
clf.named_steps['LR'].coef_

array([[ 0.22991145, -0.22981372, -1.33141786,  1.33152004]])

In [69]:
print(
f"""
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)


KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.4948252308285958
KL Score in testing data: 0.5736330128092295



In [49]:
def train_test_df_split(feature_label):
    train_data = feature_label.loc[(feature_label['Season']<2011) *
                                   (feature_label['GameType']=='T'), :]
    test_data = feature_label.loc[(feature_label['Season']>=2011) &
                                  (feature_label['GameType']=='T'), :]
    return train_data, test_data


def train_test_split(feature_label):
    train_df, test_df = train_test_df_split(feature_label)
    X_train = train_df[['Team1_season_score','Team2_season_score','Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_train = train_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values
    X_test = test_df[['Team1_season_score','Team2_season_score','Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_test = test_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values    
    return X_train, X_test, y_train, y_test

In [50]:
X_train, X_test, y_train, y_test = train_test_split(feature_label)

c:\app\conda\envs\p37web\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [61]:
clf = pipeline.Pipeline([('SS', preprocessing.StandardScaler()),
                ('LR', LogisticRegressionCV(fit_intercept=False,scoring =  'neg_log_loss'))])
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Pipeline(memory=None,
     steps=[('SS', StandardScaler(copy=True, with_mean=True, with_std=True)), ('LR', LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='neg_log_loss',
           solver='lbfgs', tol=0.0001, verbose=0))])

In [63]:
print(
f"""
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)


KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.49633304853756893
KL Score in testing data: 0.5769814327460678



# Rank Only

In [53]:
def train_test_df_split(feature_label):
    train_data = feature_label.loc[(feature_label['Season']<2011) |
                                   (feature_label['GameType']=='R'), :]
    test_data = feature_label.loc[(feature_label['Season']>=2011) &
                                  (feature_label['GameType']=='T'), :]
    return train_data, test_data


def train_test_split(feature_label):
    train_df, test_df = train_test_df_split(feature_label)
    X_train = train_df[['Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_train = train_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values
    X_test = test_df[['Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_test = test_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values    
    return X_train, X_test, y_train, y_test

In [54]:
X_train, X_test, y_train, y_test = train_test_split(feature_label)

In [55]:
train_data, test_data = train_test_df_split(feature_label)

In [56]:
train_data.corr()

,Season,TeamID1,TeamID2,Team1_season_score,Team1_OrdinalRank,Team2_season_score,Team2_OrdinalRank
Season,1.000000,-0.000083,-0.000083,-0.007960,0.047904,-0.007960,0.047904
TeamID1,-0.000083,1.000000,0.004305,0.066632,-0.092424,0.028389,-0.047744
TeamID2,-0.000083,0.004305,1.000000,0.028389,-0.047744,0.066632,-0.092424
Team1_season_score,-0.007960,0.066632,0.028389,1.000000,-0.871584,0.110839,-0.216249
Team1_OrdinalRank,0.047904,-0.092424,-0.047744,-0.871584,1.000000,-0.216249,0.381897
Team2_season_score,-0.007960,0.028389,0.066632,0.110839,-0.216249,1.000000,-0.871584
Team2_OrdinalRank,0.047904,-0.047744,-0.092424,-0.216249,0.381897,-0.871584,1.000000


In [57]:
X_train.shape

(165106, 2)

In [58]:
clf = LogisticRegressionCV(fit_intercept=False,scoring =  'neg_log_loss')
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='neg_log_loss',
           solver='lbfgs', tol=0.0001, verbose=0)

In [70]:
print(
f"""`
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)

`
KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.4948252308285958
KL Score in testing data: 0.5736330128092295



In [71]:
def train_test_df_split(feature_label):
    train_data = feature_label.loc[(feature_label['Season']<2011) &
                                   (feature_label['GameType']=='T'), :]
    test_data = feature_label.loc[(feature_label['Season']>=2011) &
                                  (feature_label['GameType']=='T'), :]
    return train_data, test_data


def train_test_split(feature_label):
    train_df, test_df = train_test_df_split(feature_label)
    X_train = train_df[['Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_train = train_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values
    X_test = test_df[['Team1_OrdinalRank','Team2_OrdinalRank']].values
    y_test = test_df['Winner'].apply(lambda x: 1 if x=='Team1' else 0).values    
    return X_train, X_test, y_train, y_test

In [72]:
X_train, X_test, y_train, y_test = train_test_split(feature_label)

In [73]:
X_train

array([[241.37      , 236.2175    ],
       [241.37      ,   9.41627907],
       [ 31.08726415,   9.41627907],
       ...,
       [  3.95454545, 146.05320814],
       [ 38.87575758, 113.44444444],
       [ 16.92898551, 109.01095462]])

In [74]:
X_train.shape

(1024, 2)

In [75]:
clf = LogisticRegressionCV(fit_intercept=False,scoring =  'neg_log_loss')
clf.fit(X_train, y_train)

c:\app\conda\envs\p37web\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring='neg_log_loss',
           solver='lbfgs', tol=0.0001, verbose=0)

In [76]:
print(
f"""`
KL Score for dummy estimator: {KL_score(y_test, np.repeat(0.5, len(y_test)))}
KL Score in training data: {KL_score(y_train, predict_prob(clf, X_train))}
KL Score in testing data: {KL_score(y_test, predict_prob(clf, X_test))}
"""
)

`
KL Score for dummy estimator: 0.6931471805599453
KL Score in training data: 0.5309060958025951
KL Score in testing data: 0.5855290376895113



## Observation
The performance in rank only seems to be the same as combining diff score with rank, and is better than diff score only. We now check the correlation between rank and diff-score, which shows a very strong correlation. 

In [77]:
score_rank[['relative_score','OrdinalRank']].corr()

,relative_score,OrdinalRank
relative_score,1.00000,-0.86982
OrdinalRank,-0.86982,1.00000


In [78]:
score_rank[['relative_score','OrdinalRank']].corr('spearman')

,relative_score,OrdinalRank
relative_score,1.00000,-0.88029
OrdinalRank,-0.88029,1.00000
